# Basic Example of the Pogit Model

This notebook shows you how to fit a basic Pogit model with one covariate for the event generating process `lambda` and one covariate for the reporting rate `p`.

After understanding the basic modeling setup in this notebook, see the `Regularizer-And-Constraint-Demos` notebook for examples of regularization and constraints that can improve the fit to `p` and `lambda`. 

See `Road-Injuries-Tutorial` for an example of these methods applied to realistic data with additional covariates, and to see the effect of overdispersion and model misspecification on the model fit. The road injuries tutorial also addresses modeling data where each observation has a different sample size, in which case the model must include an offset term.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from xspline import XSpline
from regmod.data import Data
from regmod.variable import Variable, SplineVariable
from regmod.prior import SplineUniformPrior, SplineGaussianPrior, LinearGaussianPrior
from regmod.models import PogitModel
from regmod.utils import SplineSpecs
from regmod.optimizer import scipy_optimize

In [ ]:
def logit(p):
    return np.log(p/(1-p))

In [ ]:
# Global plotting parameters
plt.rc('font', size=16) #controls default text size
plt.rc('axes', titlesize=20) #fontsize of the title
plt.rc('axes', labelsize=16) #fontsize of the x and y labels
plt.rc('xtick', labelsize=14) #fontsize of the x tick labels
plt.rc('ytick', labelsize=14) #fontsize of the y tick labels
plt.rc('legend', fontsize=14) #fontsize of the legend

In [ ]:
np.random.seed(123)
NUM_OBS = 500

## Generate Data

Generate data according to `logit(p) = -sin(2\pi x0)` and `lambda = 15 + exp(cos(2\pi x1))` with `x0, x1 ~ Uniform(0,1)`

In this notation, `x0` and `x1` are covariates. `n` is the total number of events (observed and unobserved) so that

`n ~ Poisson(lambda)`

while `y` is the number of observed events

`y ~ Binomial(n, p)`

In [ ]:
def generateData():
    x0 = np.random.rand(NUM_OBS)
    x1 = np.random.rand(NUM_OBS)

    true_p = 1.0/(1.0 + np.exp(-np.sin(x0*2.0*np.pi)))
    true_lam = 15.0 + np.exp(np.cos(x1*2.0*np.pi))
    n = np.random.poisson(true_lam)
    y = np.random.binomial(n=n, p=true_p)
    
    return x0, x1, y, n

In [ ]:
x0, x1, y, n = generateData()

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(10, 5*3))
ax[0].scatter(x0, y/n, marker="x", color="gray", alpha=0.5, label="y/n (unobserved)")
ax[0].plot(np.linspace(0, 1, 100), 1.0/(1.0 + np.exp(-np.sin(np.linspace(0, 1, 100)*2.0*np.pi))), 
           color="#DC143C", label="True p", linestyle="--")
ax[0].set_xlabel("x0")
ax[0].set_ylabel("p")
ax[0].set_title("Data and Generating Model", loc="left", size=20)
leg = ax[0].legend()
for marker in leg.legendHandles:
    marker.set_alpha(1)

ax[1].scatter(x1, n, marker="x", color="gray", alpha=0.5, label="n (unobserved)")
ax[1].plot(np.linspace(0, 1, 100), 15.0 + np.exp(np.cos(np.linspace(0, 1, 100)*2.0*np.pi)), 
               color="#DC143C", label=r'True $\lambda$', linestyle="--")
ax[1].set_ylim()
ax[1].set_xlabel("x1")
ax[1].set_ylabel(r'$\lambda$')
leg = ax[1].legend()
for marker in leg.legendHandles:
    marker.set_alpha(1)

ax[2].scatter(x0, y, marker=".", color="gray", label="y (observed)")
ax[2].plot(np.linspace(0, 1, 100), (15.0 + np.exp(np.cos(np.linspace(0, 1, 100)*2.0*np.pi)))/(1.0 + np.exp(-np.sin(np.linspace(0, 1, 100)*2.0*np.pi))), 
           color="#DC143C", label=r'True $\mu=\lambda p$', linestyle="--")
ax[2].set_xlabel("x0, x1")
ax[2].set_ylabel(r'$\mu=\lambda p$')
leg = ax[2].legend()
for marker in leg.legendHandles:
    marker.set_alpha(1)
    
plt.tight_layout()
plt.show()

## Fit the Pogit model to the observations

Model both `p` and `lambda` by second-degree splines, with two knots for `p` and one knot for `lambda`

In [ ]:
var0 = SplineVariable(name="x0",
                          spline_specs=SplineSpecs(knots=np.array([0.0, 0.25, 0.75, 1.0]),
                                                   knots_type="abs",
                                                   degree=3))

var1 = SplineVariable(name="x1",
                          spline_specs=SplineSpecs(knots=np.array([0.0, 0.5, 1.0]),
                                                   knots_type="abs",
                                                   degree=3))

Fit the Pogit model using the Regmod framework

In [ ]:
# Build and fit model
df = pd.DataFrame({"y": y, "x0": x0, "x1": x1})
data = Data(col_obs="y", col_covs=["x0", "x1"], df=df)
model = PogitModel(data, param_specs={"p": {"variables": [var0]}, "lam": {"variables": [var1]}})
result = scipy_optimize(model)

# Record the estimated pHat and lambdaHat at every value of x0, x1
coefs = model.split_coefs(result["coefs"])

Plot the model predictions over the range `[0,1]` in both `x0` and `x1`

In [ ]:
# Predict p, lambda and mu over the entire x0 and x1 range, for plotting
df_pred = pd.DataFrame({"x0": np.linspace(0, 1, 100), "x1": np.linspace(0, 1, 100)})
data_pred = Data(col_covs=["x0", "x1"], df=df_pred)

pred0 = model.params[0].get_param(coefs[0], data_pred)
pred1 = model.params[1].get_param(coefs[1], data_pred)

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(10, 5*3))
ax[0].plot(np.linspace(0, 1, 100), pred0, color="#008080", label="Model fit")
ax[0].scatter(x0, y/n, marker="x", color="gray", alpha=0.5, label="y/n (unobserved)")
ax[0].plot(np.linspace(0, 1, 100), 1.0/(1.0 + np.exp(-np.sin(np.linspace(0, 1, 100)*2.0*np.pi))), 
           color="#DC143C", label="True p", linestyle="--")
ax[0].set_xlabel("x0")
ax[0].set_ylabel("p")
ax[0].set_title("Pogit Model", loc="left")
leg = ax[0].legend()
for marker in leg.legendHandles:
    marker.set_alpha(1)

ax[1].plot(np.linspace(0, 1, 100), pred1, color="#008080", label="Model fit")
ax[1].scatter(x1, n, marker="x", color="gray", alpha=0.5, label="n (unobserved)")
ax[1].plot(np.linspace(0, 1, 100), 15.0 + np.exp(np.cos(np.linspace(0, 1, 100)*2.0*np.pi)), 
               color="#DC143C", label=r'True $\lambda$', linestyle="--")
ax[1].set_ylim()
ax[1].set_xlabel("x1")
ax[1].set_ylabel(r'$\lambda$')
leg = ax[1].legend()
for marker in leg.legendHandles:
    marker.set_alpha(1)

ax[2].plot(np.linspace(0, 1, 100), pred0*pred1, color="#008080", label="Model fit")
ax[2].scatter(x0, y, marker=".", color="gray", label="y (observed)")
ax[2].plot(np.linspace(0, 1, 100), (15.0 + np.exp(np.cos(np.linspace(0, 1, 100)*2.0*np.pi)))/(1.0 + np.exp(-np.sin(np.linspace(0, 1, 100)*2.0*np.pi))), 
           color="#DC143C", label=r'True $\mu=\lambda p$', linestyle="--")
ax[2].set_xlabel("x0, x1")
ax[2].set_ylabel(r'$\mu=\lambda p$')
leg = ax[2].legend()
for marker in leg.legendHandles:
    marker.set_alpha(1)

plt.show()

## Quantify uncertainty via 1,000 draws

Our model fitting process produces both coefficient estimates and the covariance matrix on those estimates, via sandwich estimation. We will take 1,000 draws of coefficients according to this covariance matrix, and use this to quantify uncertainty.

In [ ]:
numDraws = 1000

# Get our thousand draws, according to the covariance
thousandCoefs = np.random.multivariate_normal(mean=result["coefs"], cov=result["vcov"], size=numDraws)

The `thousandCoefs` above defines a thousand realizations of our model. We can plot the uncertainty in the estimated `p`, `lambda` and `mu` by computing these three quantities for each of the thousand draws, on the interval `[0,1]` for `x0` and `x1`.

In [ ]:
# Turn these thousand parameter draws into a thousand mu_i's, p_i's and lambda_i's for every row in processed_data
ps = np.array([model.params[0].get_param(model.split_coefs(c)[0], data_pred) for c in thousandCoefs]).T
ls = np.array([model.params[1].get_param(model.split_coefs(c)[1], data_pred) for c in thousandCoefs]).T
mus = ps*ls

In [ ]:
# Generate 1-alpha confidence intervals
alpha = 0.05

# sort each row (each observation i)
sortedLams = np.sort(ls, axis=1)
sortedPs = np.sort(ps, axis=1)
sortedMus = np.sort(mus, axis=1)

lci = int(np.floor(numDraws * alpha / 2))
uci = int(np.ceil(numDraws * (1 - alpha / 2)))

empiricalBounds = {"lamLCB": sortedLams[:, lci],
            "lamUCB": sortedLams[:, uci],
            "pLCB": sortedPs[:, lci],
            "pUCB": sortedPs[:, uci],
            "muLCB": sortedMus[:, lci],
            "muUCB": sortedMus[:, uci]}

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(20, 5*3))
for i, uqMethod, bounds in [(0, "One Thousand Draws of the Model Coefficients", None),
                           (1, "Empirical CIs from 1,000 Draws", empiricalBounds)]:
    ax[0, i].plot(np.linspace(0, 1, 100), pred0, color="#008080", label="Model fit")
    ax[0, i].scatter(x0, y/n, marker="x", color="gray", alpha=0.5, label="y/n (unobserved)")
    ax[0, i].plot(np.linspace(0, 1, 100), 1.0/(1.0 + np.exp(-np.sin(np.linspace(0, 1, 100)*2.0*np.pi))), 
               color="#DC143C", label="True p", linestyle="--")
    if bounds is None:
        # Plot the actual thousand draws
        ax[0, i].plot(np.linspace(0, 1, 100), ps, color="#008080", label=["One draw"] + ["_nolabel_"]*(numDraws-1), alpha=0.02)
    else:
        # Plot the bounds given by the UQ method
        ax[0, i].fill_between(np.linspace(0, 1, 100), bounds["pLCB"], bounds["pUCB"],
                                    facecolor="#008080", alpha=0.1)
    ax[0, i].set_xlabel("x0")
    ax[0, i].set_ylabel("p")
    ax[0, i].set_title(uqMethod, loc="left")
    leg = ax[0, i].legend()
    for marker in leg.legendHandles:
        marker.set_alpha(1)

    ax[1, i].semilogy(np.linspace(0, 1, 100), pred1, color="#008080", label="Model fit")
    ax[1, i].scatter(x1, n, marker="x", color="gray", alpha=0.5, label="n (unobserved)")
    ax[1, i].semilogy(np.linspace(0, 1, 100), 15.0 + np.exp(np.cos(np.linspace(0, 1, 100)*2.0*np.pi)), 
                   color="#DC143C", label=r'True $\lambda$', linestyle="--")
    if bounds is None:
        ax[1, i].plot(np.linspace(0, 1, 100), ls, color="#008080", label=["One draw"] + ["_nolabel_"]*(numDraws-1), alpha=0.02)
    else:
        ax[1, i].fill_between(np.linspace(0, 1, 100), bounds["lamLCB"], bounds["lamUCB"],
                                    facecolor="#008080", alpha=0.1)
    ax[1, i].set_ylim()
    ax[1, i].set_xlabel("x1")
    ax[1, i].set_ylabel(r'$\lambda$')
    leg = ax[1, i].legend()
    for marker in leg.legendHandles:
        marker.set_alpha(1)

    ax[2, i].plot(np.linspace(0, 1, 100), pred0*pred1, color="#008080", label="Model fit")
    ax[2, i].scatter(x0, y, marker=".", color="gray", label="y (observed)")
    ax[2, i].plot(np.linspace(0, 1, 100), (15.0 + np.exp(np.cos(np.linspace(0, 1, 100)*2.0*np.pi)))/(1.0 + np.exp(-np.sin(np.linspace(0, 1, 100)*2.0*np.pi))), 
               color="#DC143C", label=r'True $\mu=\lambda p$', linestyle="--")
    if bounds is None:
        ax[2, i].plot(np.linspace(0, 1, 100), mus, color="#008080", label=["One draw"] + ["_nolabel_"]*(numDraws-1), alpha=0.02)
    else:
        ax[2, i].fill_between(np.linspace(0, 1, 100), bounds["muLCB"], bounds["muUCB"],
                                    facecolor="#008080", alpha=0.1)
    ax[2, i].set_xlabel("x0, x1")
    ax[2, i].set_ylabel(r'$\mu=\lambda p$')
    leg = ax[2, i].legend()
    for marker in leg.legendHandles:
        marker.set_alpha(1)

plt.show()